# FRED & Yahoo Data Fetch — Debug Notebook

This notebook downloads monthly data from **Yahoo Finance** (prices) and **FRED** (CPI/T-Bills),
prints sample rows (head/tail), and shows basic stats. It also saves CSVs in the same folder.

> If you're behind a proxy or on a VPN that rewrites HTTPS, the FRED cell uses a proxy-proof
> approach. If needed, you can also **unset proxy environment variables** in the next cell.

In [1]:
# OPTIONAL: Uncomment to ignore proxy environment variables for this session
# import os
# for k in ["HTTP_PROXY","HTTPS_PROXY","http_proxy","https_proxy","ALL_PROXY","all_proxy","NO_PROXY","no_proxy"]:
#     os.environ.pop(k, None)
# print("Proxy env vars cleared for this kernel session (if uncommented).")

In [2]:
import io
import os
from datetime import datetime

import pandas as pd

def log(msg: str):
    print(f"[debug] {msg}")

In [3]:
def fetch_fred_csv(series_id: str, start: str | None = None, end: str | None = None) -> pd.DataFrame:
    """
    Fetch a FRED series using CSV endpoints (no API key). We:
    - ignore proxy env vars
    - try 2 official CSV forms + a last-resort CSV API
    - verify content-type is CSV (or plain text)
    - tolerate different date/value column names

    Returns a DataFrame indexed by date with a single column = series_id,
    resampled to month-end frequency.
    """
    import requests, certifi

    urls = [
        f"https://fred.stlouisfed.org/series/{series_id}/downloaddata/{series_id}&frequency=m.csv",
        f"https://fred.stlouisfed.org/graph/fredgraph.csv?id={series_id}&frequency=m",
        f"https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&frequency=m&file_type=csv",
    ]

    sess = requests.Session()
    sess.trust_env = False  # ignore system proxies
    headers = {"User-Agent": "portfolio-tester-debug/0.1"}

    last_exc = None
    for url in urls:
        try:
            log(f"GET {url}")
            r = sess.get(
                url,
                timeout=30,
                verify=certifi.where(),
                headers=headers,
                allow_redirects=True,
                proxies={"http": None, "https": None},
            )
            ctype = r.headers.get("Content-Type", "")
            log(f"  -> HTTP {r.status_code} Content-Type={ctype!r}")
            r.raise_for_status()

            if "csv" not in ctype.lower() and "text" not in ctype.lower():
                preview = r.text[:200].replace("\n", "\n")
                raise RuntimeError(f"Unexpected content type: {ctype}; preview='{preview}...'")

            df = pd.read_csv(io.StringIO(r.text))
            cols_lower = {c.lower(): c for c in df.columns}
            date_col = cols_lower.get("date") or cols_lower.get("observation_date")
            if date_col is None:
                preview = r.text.splitlines()[:5]
                raise RuntimeError("CSV missing a DATE/observation_date column. First lines: " + " | ".join(preview))

            if series_id in df.columns:
                val_col = series_id
            else:
                non_date = [c for c in df.columns if c != date_col]
                if not non_date:
                    raise RuntimeError("CSV missing value column.")
                val_col = non_date[0]
                df = df.rename(columns={val_col: series_id})

            df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
            df = df.dropna(subset=[date_col]).set_index(date_col)
            df[series_id] = pd.to_numeric(df[series_id], errors="coerce")
            df = df.dropna(subset=[series_id])

            if start:
                df = df[df.index >= pd.to_datetime(start)]
            if end:
                df = df[df.index <= pd.to_datetime(end)]

            df = df.resample("ME").last()
            return df[[series_id]]
        except Exception as e:
            last_exc = e
            log(f"  !! Attempt failed: {e}")

    raise RuntimeError(f"All FRED CSV attempts failed for {series_id}: {last_exc}")

In [4]:
def fetch_yahoo_prices_monthly(tickers: list[str]) -> pd.DataFrame:
    """Download daily auto-adjusted prices from Yahoo and resample to month-end."""
    import yfinance as yf

    log(f"Downloading from Yahoo Finance: {tickers}")
    data = yf.download(
        tickers,
        auto_adjust=True,
        progress=False,
        interval="1d",
        group_by="column",
        period="max",
    )

    def extract_close_frame(data, tickers):
        import pandas as pd
        if isinstance(data, pd.DataFrame) and isinstance(data.columns, pd.MultiIndex):
            lvl0 = set(data.columns.get_level_values(0))
            if "Close" in lvl0:
                return data["Close"].copy()
            if "Adj Close" in lvl0:
                return data["Adj Close"].copy()
            for fld in ("Close", "Adj Close"):
                try:
                    return data.xs(fld, level=1, axis=1).copy()
                except KeyError:
                    pass
        if isinstance(data, pd.DataFrame):
            for fld in ("Close", "Adj Close"):
                if fld in data.columns:
                    return data[[fld]].rename(columns={fld: tickers[0]}).copy()
        raise RuntimeError(f"Could not find Close/Adj Close columns. Columns={data.columns}")

    px_daily = extract_close_frame(data, tickers)
    present = [t for t in tickers if t in px_daily.columns]
    if not present:
        raise RuntimeError("None of the requested tickers returned price data.")
    px_daily = px_daily[present]
    monthly = px_daily.resample("ME").last().dropna(how="all")
    return monthly

In [5]:
# === PARAMETERS ===
tickers = ["VTI", "TLT", "IEF", "GSG", "GLD"]
fred_series = ["CPIAUCSL", "TB3MS"]  # CPI and 3M T-Bill
start = None  # e.g., "1990-01-01"
end = None

In [6]:
try:
    px_m = fetch_yahoo_prices_monthly(tickers)
    display(px_m.head(10))
    display(px_m.tail(10))
    print("\n--- Basic stats (Yahoo monthly prices) ---")
    display(px_m.describe(include='all'))
    px_m.to_csv("yahoo_prices_monthly.csv")
    log("Saved: yahoo_prices_monthly.csv")
except Exception as e:
    log(f"YAHOO ERROR: {e}")

[debug] Downloading from Yahoo Finance: ['VTI', 'TLT', 'IEF', 'GSG', 'GLD']


Ticker,VTI,TLT,IEF,GSG,GLD
Date,,,,,
2001-06-30,36.763496,NaN,NaN,NaN,NaN
2001-07-31,36.060776,NaN,NaN,NaN,NaN
2001-08-31,33.949303,NaN,NaN,NaN,NaN
2001-09-30,31.001394,NaN,NaN,NaN,NaN
2001-10-31,31.638060,NaN,NaN,NaN,NaN
2001-11-30,34.047646,NaN,NaN,NaN,NaN
2001-12-31,34.538651,NaN,NaN,NaN,NaN
2002-01-31,34.237144,NaN,NaN,NaN,NaN
2002-02-28,33.637417,NaN,NaN,NaN,NaN


Ticker,VTI,TLT,IEF,GSG,GLD
Date,,,,,
2025-01-31,295.835876,84.954613,90.400032,22.520000,258.559998
2025-02-28,290.248047,89.794571,92.930977,22.190001,263.269989
2025-03-31,273.253082,88.712822,93.248756,22.770000,288.140015
2025-04-30,271.244781,87.505913,94.233574,20.750000,303.769989
2025-05-31,288.206268,84.696442,93.065460,21.170000,303.600006
2025-06-30,303.087463,86.952782,94.556313,22.040001,304.829987
2025-07-31,310.028168,85.961823,93.994728,22.799999,302.959991
2025-08-31,317.317902,85.972748,95.543900,22.770000,318.070007
2025-09-30,328.170013,89.059998,96.166000,22.969999,355.470001



--- Basic stats (Yahoo monthly prices) ---


Ticker,VTI,TLT,IEF,GSG,GLD
count,293.000000,280.000000,280.000000,232.000000,252.000000
mean,101.334921,80.700416,76.407353,25.891379,134.446428
std,75.947708,27.041241,18.720253,11.815561,56.265973
min,25.391893,37.514793,41.981911,8.480000,41.650002
25%,42.812725,55.683295,58.996054,16.255000,105.702499
50%,70.872620,84.579475,82.340939,22.170000,124.404999
75%,136.447708,98.255541,89.905842,32.464999,165.532501
max,331.010010,146.904068,108.444946,74.910004,378.790009


[debug] Saved: yahoo_prices_monthly.csv


In [7]:
for sid in fred_series:
    try:
        df = fetch_fred_csv(sid, start=start, end=end)
        print(f"\n### FRED {sid} monthly — head")
        display(df.head(12))
        print(f"\n### FRED {sid} monthly — tail")
        display(df.tail(12))
        print(f"\n--- Basic stats (FRED {sid}) ---")
        display(df.describe(include='all'))
        out_path = f"fred_{sid}_monthly.csv"
        df.to_csv(out_path)
        log(f"Saved: {out_path}")
    except Exception as e:
        log(f"FRED ERROR for {sid}: {e}")

[debug] GET https://fred.stlouisfed.org/series/CPIAUCSL/downloaddata/CPIAUCSL&frequency=m.csv
[debug]   !! Attempt failed: HTTPSConnectionPool(host='https', port=443): Max retries exceeded with url: /fred.stlouisfed.org/docs/api/fred/ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x1163a6150>: Failed to resolve 'https' ([Errno 8] nodename nor servname provided, or not known)"))
[debug] GET https://fred.stlouisfed.org/graph/fredgraph.csv?id=CPIAUCSL&frequency=m
[debug]   -> HTTP 200 Content-Type='application/csv'

### FRED CPIAUCSL monthly — head


,CPIAUCSL
observation_date,
1947-01-31,21.48
1947-02-28,21.62
1947-03-31,22.00
1947-04-30,22.00
1947-05-31,21.95
1947-06-30,22.08
1947-07-31,22.23
1947-08-31,22.40
1947-09-30,22.84



### FRED CPIAUCSL monthly — tail


,CPIAUCSL
observation_date,
2024-09-30,314.851
2024-10-31,315.564
2024-11-30,316.449
2024-12-31,317.603
2025-01-31,319.086
2025-02-28,319.775
2025-03-31,319.615
2025-04-30,320.321
2025-05-31,320.580



--- Basic stats (FRED CPIAUCSL) ---


...


### FRED TB3MS monthly — tail


,TB3MS
observation_date,
2024-10-31,4.51
2024-11-30,4.42
2024-12-31,4.27
2025-01-31,4.21
2025-02-28,4.22
2025-03-31,4.20
2025-04-30,4.21
2025-05-31,4.25
2025-06-30,4.23



--- Basic stats (FRED TB3MS) ---


,TB3MS
count,1101.000000
mean,3.416966
std,3.099861
min,0.010000
25%,0.380000
50%,2.930000
75%,5.220000
max,16.300000


[debug] Saved: fred_TB3MS_monthly.csv


In [8]:
import os, glob
print("Working directory:", os.getcwd())
print("Saved CSVs:", glob.glob("*.csv"))

Working directory: /Users/riccardotrabucco/Python/portfolio-tester
Saved CSVs: ['yahoo_prices_monthly.csv', 'fred_CPIAUCSL_monthly.csv', 'fred_TB3MS_monthly.csv']
